## Connect to the Reddit API

To Connect to Reddit API, we must have :
- account (username & password)
- go to [reddit api documentation](https://www.reddit.com/dev/api/)
    - Follow the instruction to connect to the API [ access rules](https://github.com/reddit-archive/reddit/wiki/API)
    - On Rules [OAuth2](https://github.com/reddit-archive/reddit/wiki/OAuth2) we have:
        - The lik to get the web_app & secret code [https://www.reddit.com/prefs/apps](https://www.reddit.com/prefs/apps)
    - On [Quick Start Example](https://github.com/reddit-archive/reddit/wiki/OAuth2-Quick-Start-Example) we have :
        - The API url to get a token : [https://www.reddit.com/api/v1/access_token](https://www.reddit.com/api/v1/access_token)
        - An exemple how to connect with request method [Python example](https://github.com/reddit-archive/reddit/wiki/OAuth2-Quick-Start-Example#python-example)
        - The APi url after getting the token [https://oauth.reddit.com](https://oauth.reddit.com)

In [1]:
import json
from pathlib import Path

import requests.auth

In [2]:
path = Path.cwd().parent / "secret"

def open_file(file_name):
    """
    :param file_name: must be json file
    :return: load json file on variable
    """
    with open(path / file_name, "r") as file:
        return json.load(file)

user_name = open_file("user_name.json")
reddit_key = open_file("reddit_key.json")

In [3]:
user = list(reddit_key.keys())[0]
secret = list(reddit_key.values())[0]
username =list(user_name.keys())[0]
password = list(user_name.values())[0]

## Getting the token code

In [4]:
client_auth = requests.auth.HTTPBasicAuth(user,secret)
post_data={"grant_type":"password","username":username, "password":password}
headers={"User-agent":"FORMATION"}

In [5]:
response = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=client_auth, data=post_data, headers=headers)
print(response.status_code)

200


## Access to the API

In [6]:
access_token = 'bearer ' + response.json()["access_token"]

In [7]:
headers = {"authorization" : access_token, "User-agent" : "FORMATION"}
params = {"t" : "day"} # to get the last day
resp = requests.get("https://oauth.reddit.com/r/python/top", headers=headers, params=params).json()

**Save results on JSON file**

In [8]:
json_file = Path.cwd() / "python_top.json"
json_file.touch()

In [9]:
with open(json_file, "w") as f:
    json.dump(resp, f, indent=4)


## Extract the most popular article

In [10]:
file_path = Path.cwd() / "python_top.json"
with open(file_path, 'r') as f:
    python_top = json.load(f)

In [11]:
python_top_article = python_top['data']['children']
most_upvoted = ""
most_upvotes = 0
for article in python_top_article:
    ar = article['data']
    if ar['ups'] >= most_upvotes:
        most_upvoted = ar['id']
        most_upvotes = ar['ups']
print(most_upvotes,"\n", most_upvoted)
ID = most_upvoted

127 
 wr8f5u


## Extract all comments

To extract comments from reddit, we have to connect to [https://oauth.reddit.com/r/python/comments/{id}](www.reddit.com/r/python/top/comments/) and for this exemple the id = most_upvoted

In [12]:
urls = "https://oauth.reddit.com/r/python/comments/" + ID
print(urls)

comments = requests.get(urls, headers=headers).json()
with open("popular_comment.json", 'w') as f:
    json.dump(comments, f, indent=4)

https://oauth.reddit.com/r/python/comments/wr8f5u


## Get the most popular comment

In [13]:
with open('popular_comment.json', 'r') as f:
    comment = json.load(f)
python_top_article  = comment[1]['data']['children'] # data are located on second block
most_upvoted = ""
most_upvotes = 0
for article in python_top_article:
    ar = article['data']
    if ar['ups'] >= most_upvotes:
        most_upvoted = ar['id']
        most_upvotes = ar['ups']
print(most_upvotes,"\n", most_upvoted)
ids = most_upvoted
urls = "https://oauth.reddit.com/r/python/comments" +"/"+ID+"/"+ ids
print(urls)

20 
 iks2uc5
https://oauth.reddit.com/r/python/comments/wr8f5u/iks2uc5


The most popular comment is [https://oauth.reddit.com/r/python/comments/wr8f5u/iks2uc5](https://oauth.reddit.com/r/python/comments/wr8f5u/iks2uc5)